In [1]:
from IPython.display import clear_output
from qiskit import IBMQ
from qiskit_optimization import QuadraticProgram
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.algorithms import QAOA

In [2]:
clear_output()

## Small 1D Problem
<br>
Task: minimize  $f(q1, q2) = 8q_1/9 − q_2/9 − 4q_1q_2/9 + 1/9$

In [7]:
qp = QuadraticProgram()
qp.binary_var('q1')
qp.binary_var('q2')
qp.minimize(linear={'q1':8/9, 'q2':-1/9}, quadratic={('q1','q2'):-4/9}, constant=1/9)
print(qp.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: 0.888888888889 q1 - 0.111111111111 q2 + [ - 0.888888888889 q1*q2 ]/2 +
      0.111111111111
Subject To

Bounds
 0 <= q1 <= 1
 0 <= q2 <= 1

Binaries
 q1 q2
End



In [8]:
qins = QuantumInstance(backend=Aer.get_backend('qasm_simulator'), shots=1000, seed_simulator=123)
meo = MinimumEigenOptimizer(min_eigen_solver=QAOA(reps=1, quantum_instance=qins))
result = meo.solve(qp)
print('result:\n', result)
print('\ntime:', result.min_eigen_solver_result.optimizer_time)

result:
 optimal function value: 0.0
optimal value: [0. 1.]
status: SUCCESS

time: 0.02435612678527832


In [5]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q')